Plan de la tâche - Stakeholders' map

1. Création des types de données
2. Création des tables csv
3. Importation dans neo4j sous forme de noeuds-relations
4. Design des requêtes
5. Possibilité d'interroger la base via un notebook Jupyter


Lien du github : https://github.com/MaximeCapron/foodforneo4j

# 5. Possibilité d'interroger la base

In [1]:
import numpy as np
import pandas as pd
from py2neo import Graph
from py2neo.data import Node, Relationship

In [2]:
graph = Graph("bolt://3.90.65.37:34875", auth=("neo4j", "volumes-capes-retrievals"))

## Fonctions

In [5]:
def clear():
    # Commencer par supprimer tout ce qui préexiste pour avoir une feuille propre
    try:
        graph.run("MATCH (n) DETACH DELETE n")
    except:
        pass

In [57]:
def positions_projet(position="toutes"):
    if(position=="toutes"):
        query = """MATCH (p:Personne)-[rel:EMPLOYE_DANS]-(c:Entreprise)
        WHERE exists(p.Position)
        RETURN p.Nom AS nom, c.Titre AS entreprise, rel.position AS rôle, 
        p.Position AS position ORDER BY position"""
    elif(position==1):
        query = """MATCH (p:Personne)-[rel:EMPLOYE_DANS]-(c:Entreprise)
        WHERE p.Position = "Avocat"
        RETURN p.Nom AS nom, c.Titre AS entreprise, rel.position AS rôle, 
        p.Position AS position ORDER BY rôle"""
    elif(position==0):
        query = """MATCH (p:Personne)-[rel:EMPLOYE_DANS]-(c:Entreprise)
        WHERE p.Position = "Neutre"
        RETURN p.Nom AS nom, c.Titre AS entreprise, rel.position AS rôle, 
        p.Position AS position ORDER BY rôle"""
    elif(position==-1):
        query = """MATCH (p:Personne)-[rel:EMPLOYE_DANS]-(c:Entreprise)
        WHERE p.Position = "A convaincre"
        RETURN p.Nom AS nom, c.Titre AS entreprise, rel.position AS rôle, 
        p.Position AS position ORDER BY rôle"""
    else:
        raise "Position inconnue"
    return graph.run(query).to_data_frame()

In [8]:
def distance_pos(A,B):
    query = """
    MATCH (a:Personne)-[:EMPLOYE_DANS {position:'"""+A[0]+"""'}]-(:Entreprise {Titre:'"""+A[1]+"""'})
    MATCH (b:Personne)-[:EMPLOYE_DANS {position:'"""+B[0]+"""'}]-(:Entreprise {Titre:'"""+B[1]+"""'})
    MATCH p = shortestPath((a)-[*]-(b))
    RETURN a.Nom,b.Nom,length(p) AS distance"""
    return graph.run(query).to_data_frame()

In [48]:
def distance_noms(nomA,nomB):
    query = "MATCH p = shortestPath((a:Personne {Nom:'"+nomA+"'})-[*]-(b:Personne {Nom:'"+nomB+"'})) RETURN a.Nom, b.Nom, length(p) AS distance"
    return graph.run(query).to_data_frame()

In [9]:
def a_connaitre():
    query = """
    MATCH (a:Entreprise)-[:EMPLOYE_DANS]-(p:Personne)
    WHERE NOT a.Titre = "OBS"
    WITH p
    MATCH (c:Entreprise {Titre:"OBS"})
    MATCH short=shortestpath((c)-[*]-(p))
    WITH p,length(short) AS len
    WHERE len > 2
    RETURN p.Nom AS nom
    """
    return graph.run(query).to_data_frame()

In [11]:
connue = lambda x : not x in a_connaitre()["nom"].values

In [81]:
def bilan_de_situation(cas):
    A = positions_projet()
    A = A[A["entreprise"]==cas]
    A["connue"] = A["nom"].apply(connue)
    return A.drop("entreprise",axis=1)

In [84]:
def alertes(cas):
    a = bilan_de_situation(cas)
    a = a[a["connue"]==False]
    a = a[a["position"]=="A convaincre"]
    return a[["nom","rôle"]]

# Appels

In [42]:
clear()

In [43]:
#Entreprises
OBS = Node("Entreprise",Titre="OBS",index=11)
EDF = Node("Entreprise",Titre="EDF",index=12)

#Employés d'OBS
Jeanne = Node("Personne",Nom="Jeanne",Position="",index=1)
Charles = Node("Personne",Nom="Charles",Position="",index=2)
François = Node("Personne",Nom="François",Position="",index=3)

#Employés d'EDF
Eugénie = Node("Personne",Nom="Eugénie",Position="Avocat",index=4)
Marcel = Node("Personne",Nom="Marcel",Position="A convaincre",index=5)
Léon = Node("Personne",Nom="Léon",Position="Avocat",index=6)
Hélène = Node("Personne",Nom="Helène",Position="Avocat",index=7)
Henriette = Node("Personne",Nom="Henriette",Position="Neutre",index=8)
Jules = Node("Personne",Nom="Jules",Position="Avocat",index=9)
Pierre = Node("Personne",Nom="Pierre",Position="A convaincre",index=10)

In [44]:
for node in [OBS,EDF,Jeanne,Charles,François,Eugénie,Marcel,Léon,Hélène,Henriette,Jules,Pierre]:
    graph.create(node)

In [45]:
#Relation: EMPLOYE_DANS
graph.create(Relationship(Jeanne,"EMPLOYE_DANS",OBS,position="Commercial"))
graph.create(Relationship(Charles,"EMPLOYE_DANS",OBS,position="CEO"))
graph.create(Relationship(François,"EMPLOYE_DANS",OBS,position="CFO"))
graph.create(Relationship(Eugénie,"EMPLOYE_DANS",EDF,position="Commercial"))
graph.create(Relationship(Marcel,"EMPLOYE_DANS",EDF,position="Commercial"))
graph.create(Relationship(Léon,"EMPLOYE_DANS",EDF,position="Technique"))
graph.create(Relationship(Hélène,"EMPLOYE_DANS",EDF,position="Technique"))
graph.create(Relationship(Henriette,"EMPLOYE_DANS",EDF,position="Marketing"))
graph.create(Relationship(Jules,"EMPLOYE_DANS",EDF,position="CEO"))
graph.create(Relationship(Pierre,"EMPLOYE_DANS",EDF,position="CFO"))

#Relation: CONNAISSANCE
graph.create(Relationship(Jeanne,"CONNAISSANCE",Eugénie))
graph.create(Relationship(Jeanne,"CONNAISSANCE",Marcel))
graph.create(Relationship(Charles,"CONNAISSANCE",Henriette))
graph.create(Relationship(Charles,"CONNAISSANCE",Léon))
graph.create(Relationship(François,"CONNAISSANCE",Jules))

In [70]:
positions_projet(1)

,nom,entreprise,rôle,position
0,Jules,EDF,CEO,Avocat
1,Eugénie,EDF,Commercial,Avocat
2,Léon,EDF,Technique,Avocat
3,Helène,EDF,Technique,Avocat


In [82]:
bilan_de_situation("EDF")

,nom,rôle,position,connue
3,Marcel,Commercial,A convaincre,True
4,Pierre,CFO,A convaincre,False
5,Eugénie,Commercial,Avocat,True
6,Léon,Technique,Avocat,True
7,Helène,Technique,Avocat,False
8,Jules,CEO,Avocat,True
9,Henriette,Marketing,Neutre,True


In [85]:
alertes("EDF")

,nom,rôle
4,Pierre,CFO


In [100]:
Pierre_EDF = ["CFO","EDF"]
Charles_OBS = ["CEO","OBS"]
distance_pos(Pierre_EDF,Charles_OBS)

,a.Nom,b.Nom,distance
0,Pierre,Charles,3
